In [ ]:
from langchain_community.document_loaders import DirectoryLoader,PyMuPDFLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_groq import ChatGroq

from langchain_core.runnables import RunnableParallel,RunnablePassthrough,RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import ContextualCompressionRetriever,EnsembleRetriever

from langchain_cohere import CohereRerank
from dotenv import load_dotenv
load_dotenv()


True

In [88]:
callback = [StreamingStdOutCallbackHandler()]
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
rerank_model = "rerank-english-v3.0"
# model = ChatGroq(model="gemma2-9b-it",streaming = True,callbacks=callback)
model = ChatGroq(model="deepseek-r1-distill-llama-70b",streaming = True,callbacks=callback)

In [65]:
def document_load(path):
    "This function load the Document"
    loader = DirectoryLoader(path=path,glob="*.pdf",loader_cls=PyMuPDFLoader) #It does not load PDFs in subdirectories unless you use a recursive loader or modify the glob pattern (e.g., "**/*.pdf" with recursive=True).
    return loader.load()

electrical = document_load("data/Electric")
ai = document_load("data/AI")

print("length of electric ",len(electrical))
print("length of AI ",len(ai))

length of electric  87
length of AI  279


In [66]:
def document_splitting(file):
    splitter = RecursiveCharacterTextSplitter(chunk_size=800,chunk_overlap=180)
    return splitter.split_documents(file)

chunks_electrical = document_splitting(electrical)
chunks_ai = document_splitting(ai)

print("length of ",len(chunks_electrical))
print("length of ",len(chunks_ai))

length of  266
length of  827


In [67]:
def vector_store_creation(chunks,path):
    vector_store = FAISS.from_documents(
        chunks,
        embedding_model
        )
    vector_store.save_local(path)
    return vector_store

vector_store_electrical = vector_store_creation(chunks_electrical,"FAISS/electical/")
vector_store_ai = vector_store_creation(chunks_ai,"FAISS/ai/")

print("Vector store successfully created")

Vector store successfully created


In [68]:
vector_store_electrical = FAISS.load_local(embeddings=embedding_model,allow_dangerous_deserialization=True,folder_path="FAISS/electical/") 
electrical_retriever = vector_store_electrical.as_retriever(search_type="mmr",search_kwargs={"k":4})

In [69]:
vector_store_ai = FAISS.load_local(embeddings=embedding_model,allow_dangerous_deserialization=True,folder_path = "FAISS/ai/") 
ai_retriever = vector_store_ai.as_retriever(search_type="similarity",search_kwargs={"k":5})

In [70]:
bm25_retriever_electrical = BM25Retriever.from_documents(chunks_electrical)
bm25_retriever_electrical.k=5

bm25_retriever_ai = BM25Retriever.from_documents(chunks_ai)
bm25_retriever_ai.k=5

In [71]:
hybrid_retriever_electrical = EnsembleRetriever(
    retrievers=[electrical_retriever,bm25_retriever_electrical],
    weight=[0.7,0.3]
)

hybrid_retriever_ai = EnsembleRetriever(
    retrievers=[ai_retriever,bm25_retriever_ai],
    weight=[0.65,0.35]
)

In [100]:
prompt = PromptTemplate(
    template="""
You are an AI interview assistant. Answer the question strictly based on the provided context anser the quesion in points.

- If the context does not contain enough information to answer the question, respond with: "Context not enough."
- Do not use any external knowledge.


CONTEXT:
{context}

QUESTION:
{question}


ANSWER:
""",
    input_variables=["context", "question"]
)

parser = StrOutputParser()

In [101]:
def cleaning(documents):
    return "\n\n".join(doc.page_content for doc in documents)

In [102]:
rerank = CohereRerank(model=rerank_model)

In [103]:
final_retriever = ContextualCompressionRetriever(
    base_compressor=rerank,
    base_retriever=hybrid_retriever_ai
    )

In [104]:
parallel_chain = RunnableParallel({
    "context": final_retriever | RunnableLambda(cleaning),
    "question":RunnablePassthrough()
})

chain = parallel_chain | prompt | model | parser

In [105]:
from IPython.display import display,Markdown
result = chain.invoke("Explain Generative AI")
display(Markdown(result))

<think>
Okay, I need to answer the question "Explain Generative AI" based on the provided context. Let me go through the context step by step to gather the necessary information.

First, I see that Generative AI is defined as a class of AI models that can generate new data or content based on their training data. This includes text, images, audio, etc. That's a good starting point.

Looking further, the context mentions the evolution of Generative AI. It starts from the 1950s to 2000s with basic text generation and rule-based systems. Then, in the 2010s, neural networks and deep learning advancements like RNNs and LSTMs emerged. The transformer architecture in 2017 was a big leap, and since 2018, models like GPT have become increasingly powerful.

There's also a part that distinguishes Generative AI from other types of AI. It says that while discriminative models classify data, generative models create new instances. Examples include text generation, image synthesis, and music composit

<think>
Okay, I need to answer the question "Explain Generative AI" based on the provided context. Let me go through the context step by step to gather the necessary information.

First, I see that Generative AI is defined as a class of AI models that can generate new data or content based on their training data. This includes text, images, audio, etc. That's a good starting point.

Looking further, the context mentions the evolution of Generative AI. It starts from the 1950s to 2000s with basic text generation and rule-based systems. Then, in the 2010s, neural networks and deep learning advancements like RNNs and LSTMs emerged. The transformer architecture in 2017 was a big leap, and since 2018, models like GPT have become increasingly powerful.

There's also a part that distinguishes Generative AI from other types of AI. It says that while discriminative models classify data, generative models create new instances. Examples include text generation, image synthesis, and music composition.

Additionally, the hierarchy of AI is explained: AI is the broad field, Machine Learning is a subset focused on learning from data, and Deep Learning is a subset of ML using neural networks. Generative AI is a specialized branch within this hierarchy.

Putting this together, I should structure the answer to include the definition, capabilities, evolution, how it differs from other AI types, and its place within the AI hierarchy. I'll organize this into clear points for a concise explanation.
</think>

Generative AI is a specialized branch of artificial intelligence that focuses on creating new content, such as text, images, and audio. It operates by generating new data samples that resemble the data it was trained on. Here's an organized explanation:

1. **Definition and Capabilities**:
   - Generative AI models can generate new data or content based on their training data.
   - Capabilities include text generation, image synthesis, and music composition.

2. **Evolution**:
   - **Early Developments (1950s-2000s)**: Basic text generation using rule-based systems.
   - **2010s**: Advances in neural networks, including RNNs and LSTMs.
   - **2017**: Introduction of the transformer architecture, revolutionizing NLP.
   - **2018-Present**: Emergence of powerful models like GPT.

3. **Distinction from Other AI Types**:
   - Unlike discriminative models, which classify data, generative models create new instances.

4. **Hierarchy in AI**:
   - Part of a hierarchy where AI is the broad field, Machine Learning is a subset focused on learning from data, and Deep Learning uses neural networks.

This structured approach provides a clear and concise explanation of Generative AI.

In [91]:
chain.invoke("explain generative AI in 10 lines")

<think>
Alright, I need to explain Generative AI in 10 lines based on the provided context. Let me go through the context step by step to gather all the relevant points.

First, the context defines Generative AI as models that generate new data samples similar to their training data. It contrasts this with discriminative models, which classify data. Examples given include text generation, image synthesis, and music composition.

Next, the context mentions GANs and VAEs, but since the question is about explaining Generative AI in general, I might not need to go into detail on these unless it's necessary for a 10-line explanation.

The context also provides an overview of the evolution of Generative AI, starting from early developments in the 1950s with rule-based systems, moving through neural networks in the 2010s, the introduction of Transformers in 2017, and GPT models from 2018 onwards. This timeline could be useful to include.

Additionally, the context explains that Generative AI 

"<think>\nAlright, I need to explain Generative AI in 10 lines based on the provided context. Let me go through the context step by step to gather all the relevant points.\n\nFirst, the context defines Generative AI as models that generate new data samples similar to their training data. It contrasts this with discriminative models, which classify data. Examples given include text generation, image synthesis, and music composition.\n\nNext, the context mentions GANs and VAEs, but since the question is about explaining Generative AI in general, I might not need to go into detail on these unless it's necessary for a 10-line explanation.\n\nThe context also provides an overview of the evolution of Generative AI, starting from early developments in the 1950s with rule-based systems, moving through neural networks in the 2010s, the introduction of Transformers in 2017, and GPT models from 2018 onwards. This timeline could be useful to include.\n\nAdditionally, the context explains that Gene

In [92]:
chain.invoke(" What is Python? What are the benefits of using Python?")

<think>
Okay, I need to answer the question: "What is Python? What are the benefits of using Python?" using the provided context.

Looking at the context, the first question is exactly the same. So I can directly take the answer from there.

From point 1 in the context: Python is described as a programming language with objects, modules, threads, exceptions, and automatic memory management. The benefits listed are that it's simple and easy, portable, extensible, has built-in data structures, and is open source.

I should list these points clearly, making sure to cover both the definition and the benefits as given.
</think>

Python is a programming language that includes features such as objects, modules, threads, exceptions, and automatic memory management. The benefits of using Python include its simplicity and ease of use, portability, extensibility, built-in data structures, and being open source.

'<think>\nOkay, I need to answer the question: "What is Python? What are the benefits of using Python?" using the provided context.\n\nLooking at the context, the first question is exactly the same. So I can directly take the answer from there.\n\nFrom point 1 in the context: Python is described as a programming language with objects, modules, threads, exceptions, and automatic memory management. The benefits listed are that it\'s simple and easy, portable, extensible, has built-in data structures, and is open source.\n\nI should list these points clearly, making sure to cover both the definition and the benefits as given.\n</think>\n\nPython is a programming language that includes features such as objects, modules, threads, exceptions, and automatic memory management. The benefits of using Python include its simplicity and ease of use, portability, extensibility, built-in data structures, and being open source.'